In [ ]:
!pip install piexif -q

In [ ]:
!python --version

Python 3.11.13


In [ ]:
import os
import csv
import logging
import argparse
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

from tqdm import tqdm
from PIL import Image, UnidentifiedImageError
import piexif
import shutil
from types import SimpleNamespace

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Configurazione Percorso Base del Progetto - da Drive
GDRIVE_PROJECT_PATH = Path("/content/drive/MyDrive/PROGETTO")
print(f"Percorso base del progetto impostato a: {GDRIVE_PROJECT_PATH}")


# CONFIGURAZIONE PARAMETRI GLOBALI
# (Questi non cambiano)
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_HEAD = 10
EPOCHS_FINE = 8
FRAME_INTERVAL = 1
LEARN_HEAD_LR = 1e-3
LEARN_FULL_LR = 1e-5
SHARPNESS_THRESHOLD = 80.0
FRAMES_PER_PLACE_PER_VIDEO = 5


#PERCORSI DI DEFAULT DERIVATI (DA MODIFICARE)

# Percorsi per Training, Validazione e Test
DEFAULT_TRAIN_DIR = GDRIVE_PROJECT_PATH / "dataset" / "train"
DEFAULT_VAL_DIR = GDRIVE_PROJECT_PATH / "dataset" / "val"
DEFAULT_TEST_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"

# Percorsi per l'Inferenza (puntano alla cartella 'test')
# In questo modo, quando si esegue l'inferenza, userà le immagini e i video di test.
DEFAULT_IMG_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"
DEFAULT_VID_DIR = GDRIVE_PROJECT_PATH / "dataset" / "test"

# Cartella principale dove verranno salvati tutti gli output
DEFAULT_OUTPUT = GDRIVE_PROJECT_PATH / "output"

# Percorsi derivati per l'output
DEFAULT_CSV = DEFAULT_OUTPUT / "results.csv"
PROCESSED_LOG = DEFAULT_OUTPUT / "processed.log"
BEST_FRAMES_OUTPUT_DIR = DEFAULT_OUTPUT / "best_frames"
TEMP_FRAME_EXTRACT_DIR = DEFAULT_OUTPUT / "temp_frames"


#Creazione delle Directory di Output
DEFAULT_OUTPUT.mkdir(parents=True, exist_ok=True)
BEST_FRAMES_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
TEMP_FRAME_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)


# Stampa di Verifica
# Aggiornata per includere il nuovo percorso di test
print(f"\n Percorsi Configurati ")
print(f"Training Data: {DEFAULT_TRAIN_DIR}")
print(f"Validation Data: {DEFAULT_VAL_DIR}")
print(f"Test Data: {DEFAULT_TEST_DIR}")
print(f"Input Immagini/Video (Inferenza): {DEFAULT_IMG_DIR}")
print(f"Output Generale: {DEFAULT_OUTPUT}")

Percorso base del progetto impostato a: /content/drive/MyDrive/PROGETTO

--- Percorsi Configurati ---
Training Data: /content/drive/MyDrive/PROGETTO/dataset/train
Validation Data: /content/drive/MyDrive/PROGETTO/dataset/val
Test Data: /content/drive/MyDrive/PROGETTO/dataset/test
Input Immagini/Video (Inferenza): /content/drive/MyDrive/PROGETTO/dataset/test
Output Generale: /content/drive/MyDrive/PROGETTO/output
--------------------------


In [ ]:
# Questa funzione imposta il sistema di logging per il programma.
# ( ** Il logging è superiore a `print` perché permette di avere messaggi formattati,
# con livelli di gravità diversi (INFO, WARNING, ERROR) e di scrivere
# contemporaneamente sia sulla console che su un file di log. )
# `output_dir: Path` è un'annotazione di tipo (type hint), indica che la funzione si aspetta un oggetto Path.
def setup_logging(output_dir: Path):
    # Definisce il percorso del file di log all'interno della cartella di output.
    log_file = output_dir / "run.log"

    # Questo ciclo è FONDAMENTALE in ambienti come Colab/Jupyter.
    # Se la cella viene eseguita più volte, `logging.basicConfig` non ha effetto dopo la prima volta.
    # Questo codice "resetta" il sistema di logging rimuovendo tutti i gestori (handler) esistenti,
    # garantendo che la configurazione venga riapplicata correttamente ogni volta.
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    # Configura il logging a livello base.
    logging.basicConfig(
        level=logging.INFO,  # Imposta il livello minimo di messaggi da registrare (INFO e superiori).
        format="%(asctime)s %(levelname)s: %(message)s",  # Definisce il formato dei messaggi di log.
        datefmt='%Y-%m-%d %H:%M:%S',  # Definisce il formato del timestamp.
        # Definisce dove inviare i log:
        handlers=[
            logging.FileHandler(log_file, encoding='utf-8'),  # 1: a un file di testo (run.log).
            logging.StreamHandler()  # 2: alla console/output della cella.
        ]
    )
    # Registra un primo messaggio per confermare che il logging è attivo e indicare dove si trova il file.
    logging.info(f"Logging configurato. Log salvati in: {log_file}")


# Controlla se una chiave (che rappresenta un file) è già stata processata in precedenza.
# Questo evita di ri-elaborare file inutilmente, risparmiando tempo.
# `key: str` indica che si aspetta una stringa. `-> bool` indica che restituisce un booleano (True/False).
def is_already_processed(key: str) -> bool:
    try:
        # Se il file di log dei file processati non esiste, nessun file è stato processato.
        if not PROCESSED_LOG.exists(): return False
        # Apre il file in modalità lettura ('r').
        with open(PROCESSED_LOG, 'r', encoding='utf-8') as f:
           # Usa un'espressione generatore con `any()` per efficienza.
           # `any()` si ferma non appena trova una corrispondenza, senza leggere tutto il file se non necessario.
           # `line.strip()` rimuove spazi bianchi e caratteri di a capo dalla riga letta.
           return any(key == line.strip() for line in f)
    except Exception as e:
        # Se si verifica un errore durante la lettura del file, lo registra e assume che il file non sia processato.
        logging.error(f"Errore leggendo {PROCESSED_LOG}: {e}")
        return False


# Scrive una chiave nel file di log per marcarla come "processata".
def mark_processed(key: str):
    try:
        # Apre il file in modalità 'a' (append), che aggiunge testo alla fine del file senza cancellarlo.
        with open(PROCESSED_LOG, "a", encoding='utf-8') as f:
            # Scrive la chiave seguita da un carattere di a capo.
            f.write(key + "\n")
    except Exception as e:
        # Registra eventuali errori di scrittura.
        logging.error(f"Errore scrivendo su {PROCESSED_LOG}: {e}")


# Estrae frame da un file video a un intervallo di tempo specificato.
# `-> list` indica che la funzione restituisce una lista (dei percorsi dei frame salvati).
def extract_frames(video_path: Path, frame_output_base_dir: Path, interval_sec: int) -> list:
    # Lista per memorizzare i percorsi dei frame salvati.
    saved_frames_paths = []
    # Crea una sottocartella specifica per i frame di questo video, per tenere tutto ordinato.
    # `video_path.stem` è il nome del file senza estensione.
    video_frame_dir = frame_output_base_dir / video_path.stem
    video_frame_dir.mkdir(parents=True, exist_ok=True)
    try:
        # Apre il file video usando OpenCV.
        cap = cv2.VideoCapture(str(video_path))
        # Controlla se il video è stato aperto correttamente.
        if not cap.isOpened():
            logging.error(f"Impossibile aprire il video: {video_path}")
            return []  # Restituisce una lista vuota in caso di fallimento.

        # Ottiene i fotogrammi al secondo (FPS) del video. Se fallisce, usa 30.0 come default.
        fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
        # Calcola ogni quanti frame leggerne uno, basandosi sull'intervallo in secondi.
        step = max(1, int(fps * interval_sec))
        # Inizializza i contatori.
        frame_count, saved_frame_index, total_frames = 0, 0, int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Usa `tqdm` per creare una barra di avanzamento che mostra il progresso dell'estrazione.
        with tqdm(total=total_frames, desc=f"Extracting frames from {video_path.name}") as pbar:
            while True:
                # Legge il frame successivo del video. `ret` è True se la lettura ha successo.
                ret, frame = cap.read()
                if not ret: break  # Esce dal ciclo se il video è terminato.

                # Controlla se il contatore di frame è un multiplo dello 'step' calcolato.
                if frame_count % step == 0:
                    # Costruisce il percorso del file di output per il frame.
                    # Il formato `:05d` assicura che il numero del frame abbia sempre 5 cifre (es. 00001, 00012).
                    out_file_path = video_frame_dir / f"{video_path.stem}_frame{saved_frame_index:05d}.jpg"
                    # Salva il frame come immagine JPEG con qualità 90.
                    if cv2.imwrite(str(out_file_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 90]):
                        saved_frames_paths.append(out_file_path)
                        saved_frame_index += 1

                frame_count += 1
                pbar.update(1)  # Aggiorna la barra di avanzamento.

        # Rilascia l'oggetto video per liberare le risorse.
        cap.release()
        logging.info(f"Estratti {len(saved_frames_paths)} frame da {video_path.name}")
    except Exception as e:
        logging.error(f"Errore durante estrazione frame da {video_path}: {e}")
    # Restituisce la lista dei percorsi dei frame salvati.
    return saved_frames_paths


# Aggiorna i metadati EXIF di un'immagine per includere la label predetta.
def update_image_exif(img_path: Path, label: str):
    try:
        # Apre l'immagine con Pillow.
        img = Image.open(img_path)
        # Tenta di leggere i dati EXIF esistenti. Se non ce ne sono, usa un byte string vuoto.
        exif_data = img.info.get('exif', b"")
        # Carica i dati EXIF in un dizionario. Se non ci sono dati, crea un dizionario vuoto.
        exif_dict = piexif.load(exif_data) if exif_data else {'0th': {}}
        # Si assicura che il sotto-dizionario '0th' (che contiene i tag principali) esista.
        if '0th' not in exif_dict: exif_dict['0th'] = {}
        # Imposta il campo 'ImageDescription' con la nostra etichetta. `piexif` richiede che la stringa sia codificata.
        exif_dict['0th'][piexif.ImageIFD.ImageDescription] = label.encode('utf-8')
        # Rimuove la thumbnail per evitare problemi di compatibilità e ridurre la dimensione del file.
        exif_dict['thumbnail'] = None
        # Converte di nuovo il dizionario in formato bytes, pronto per essere scritto.
        exif_bytes = piexif.dump(exif_dict)
        # Salva l'immagine sovrascrivendo quella vecchia, ma con i nuovi dati EXIF.
        img.save(img_path, exif=exif_bytes, quality=95)
        # Chiude il file immagine.
        img.close()
    except Exception as e:
        # Se qualcosa va storto (es. file corrotto, formato non supportato), registra un avviso senza bloccare lo script.
        logging.warning(f"UPDATE_EXIF: Errore per {img_path}: {e}")


# Calcola un punteggio di nitidezza per un'immagine.
# Utile per filtrare i frame mossi o sfocati.
def calculate_sharpness(image_cv2):
    try:
        # Controllo di sicurezza per evitare errori se l'immagine non è valida.
        if image_cv2 is None or image_cv2.size == 0: return 0.0
        # Converte l'immagine in scala di grigi, perché la nitidezza è legata ai contorni, non ai colori.
        gray = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)
        # Applica l'operatore Laplaciano, che evidenzia le regioni con rapidi cambi di intensità (i contorni).
        # Calcola la varianza del risultato. Un'immagine nitida avrà molti contorni forti e quindi un'alta varianza.
        # Un'immagine sfocata avrà bassa varianza.
        return cv2.Laplacian(gray, cv2.CV_64F).var()
    except Exception as e:
        # Gestisce eventuali errori durante le operazioni di OpenCV.
        logging.error(f"CALCULATE_SHARPNESS: Errore: {e}")
        return 0.0

print("Funzioni di Utilità definite.")

Funzioni di Utilità definite.


In [ ]:
# Funzione per costruire il modello EfficientNetV2
# Questa funzione definisce l'intera architettura della rete neurale che verrà addestrata.
# `num_classes: int` indica che si aspetta un intero (il numero di categorie da predire).
# `-> models.Model` indica che restituisce un oggetto Modello di Keras.
def build_model(num_classes: int) -> models.Model:
    """Costruisce un modello di classificazione basato su EfficientNetV2B0 pre-addestrato."""

    # Carica il modello EfficientNetV2B0(CNN)
    # Questo modello funge da "backbone" (spina dorsale) per l'estrazione delle feature.
    backbone = EfficientNetV2B0(
        weights="imagenet",
        include_top=False,
        input_shape=(*IMG_SIZE, 3)
    )

    # Congela tutti i pesi del backbone
    # Impostando `trainable = False`, diciamo a Keras di non aggiornare i pesi del backbone
    # durante la prima fase di addestramento. In questo modo, addestriamo solo la "testa" che aggiungiamo noi.
    backbone.trainable = False

    # Costruzione della "Testa" di Classificazione
    # Questi sono i nuovi layer che mettiamo in cima al backbone.

    # 1. Prende l'output del backbone (una mappa di feature multidimensionale).
    # 2. `GlobalAveragePooling2D` calcola la media di ogni mappa di feature, trasformandola in un singolo numero.
    #    Questo riduce drasticamente il numero di parametri e rende il modello più robusto a piccole traslazioni dell'oggetto.
    #    L'output è un vettore di feature per ogni immagine.
    x = layers.GlobalAveragePooling2D(name="gap")(backbone.output)

    # 3. `Dropout` è un'altra tecnica di regolarizzazione fondamentale. Durante il training, "spegne" casualmente
    #    il 30% dei neuroni. Questo costringe la rete a non fare troppo affidamento su singoli neuroni
    #    e a imparare rappresentazioni più distribuite e robuste.
    x = layers.Dropout(0.3, name="dropout")(x)

    # 4. `Dense` è un layer "fully-connected" standard. Questo è il nostro classificatore finale.
    #    - `num_classes`: Il numero di neuroni in output, uno per ogni classe che vogliamo predire.
    #    - `activation="softmax"`: La funzione di attivazione Softmax trasforma i punteggi grezzi (logits) del layer
    #      in un vettore di probabilità, dove la somma di tutti gli elementi è 1.
    outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)

    # Crea l'oggetto Modello finale, specificando gli input (quelli del backbone) e gli output (quelli del nostro classificatore).
    model = models.Model(backbone.input, outputs, name="efficientnetv2b0_finetuned")

    # Compilazione del Modello (configura il modello per il training)
    model.compile(
        # `optimizer`: L'algoritmo che aggiorna i pesi del modello. Adam è una scelta molto comune e robusta.
        # Usiamo il learning rate definito per la fase di training della testa.
        optimizer=optimizers.Adam(learning_rate=LEARN_HEAD_LR),

        # `loss`: La funzione di perdita. Misura quanto le previsioni del modello sono sbagliate.
        # `categorical_crossentropy` è la scelta standard per problemi di classificazione multi-classe
        # quando le etichette sono in formato one-hot (come fa `label_mode="categorical"`).
        loss="categorical_crossentropy",

        # `metrics`: Metriche da monitorare durante il training. L'accuracy (precisione) è la più comune.
        metrics=["accuracy"]
    )

    logging.info(f"Modello EfficientNetV2B0 costruito e compilato per il training della testa.")
    return model


# Pipeline di Data Augmentation
# La Data Augmentation crea nuove immagini di training al volo, applicando trasformazioni casuali.
# Questo aiuta a prevenire l'overfitting e rende il modello più robusto a variazioni nelle immagini reali.
# `tf.keras.Sequential` crea una pipeline dove i dati passano attraverso i layer in sequenza.
data_augmentation_pipeline = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),        # Specchia l'immagine orizzontalmente con una probabilità del 50%.
    layers.RandomRotation(0.15),            # Ruota l'immagine di un angolo casuale fino al 15% di 360 gradi.
    layers.RandomZoom(0.15),                # Ingrandisce o rimpicciolisce l'immagine fino al 15%.
    layers.RandomContrast(0.1),             # Modifica il contrasto in modo casuale.
    layers.RandomBrightness(0.1),           # Modifica la luminosità in modo casuale.
], name="data_augmentation")


# Funzione per creare i dataset
# Questa funzione carica le immagini da una directory, le pre-processa e le prepara in un formato
# efficiente per l'addestramento con TensorFlow (`tf.data.Dataset`).
def make_dataset(dirpath: Path, shuffle: bool, subset: str = None, validation_split: float = None, augment: bool = False):
    # Controllo preliminare per assicurarsi che la directory esista.
    if not dirpath.is_dir():
        logging.error(f"MAKE_DATASET: Directory non trovata: {dirpath}")
        return None, None # Restituisce None per segnalare l'errore.
    try:
        # Funzione di utility di Keras che fa gran parte del lavoro pesante.
        # Scansiona la directory, inferisce i nomi delle classi dalle sottocartelle,
        # e carica le immagini.
        initial_ds = tf.keras.utils.image_dataset_from_directory(
            dirpath,                      # Il percorso della cartella da cui caricare le immagini.
            labels="inferred",            # I nomi delle classi sono dedotti dai nomi delle sottocartelle.
            label_mode="categorical",     # Le etichette sono convertite in formato one-hot (es. [0, 1, 0]).
            batch_size=BATCH_SIZE,        # Raggruppa le immagini in batch.
            image_size=IMG_SIZE,          # Ridimensiona tutte le immagini a IMG_SIZE.
            shuffle=shuffle,              # Mescola i dati (importante per il training).
            seed=123,                     # Seed per la riproducibilità del mescolamento e dello split.
            validation_split=validation_split, # Frazione di dati da riservare per la validazione.
            subset=subset,                # Specifica se creare il set di 'training' o 'validation'.
            interpolation='bilinear'      # Algoritmo usato per il ridimensionamento delle immagini.
        )
        # Estrae i nomi delle classi trovati dalla funzione.
        class_names = initial_ds.class_names

        # Applica la data augmentation solo se richiesto (tipicamente solo per il set di training).
        processed_ds = initial_ds
        if augment:
            # `map` applica una funzione a ogni elemento (batch) del dataset.
            # `lambda imgs, labs:` è una funzione anonima che prende un batch di immagini e le loro etichette.
            # Applica la pipeline di augmentation solo alle immagini, lasciando le etichette invariate.
            # `num_parallel_calls=tf.data.AUTOTUNE` permette a TensorFlow di parallelizzare l'operazione per la massima efficienza.
            processed_ds = processed_ds.map(lambda imgs, labs: (data_augmentation_pipeline(imgs, training=True), labs),
                                           num_parallel_calls=tf.data.AUTOTUNE)

        # Applica la funzione di preprocessing specifica del modello (es. scalare i pixel da [0, 255] a [-1, 1]).
        # Questo passo è OBBLIGATORIO e va fatto sia per il training che per la validazione/inferenza.
        processed_ds = processed_ds.map(lambda imgs, labs: (preprocess_input(imgs), labs),
                                       num_parallel_calls=tf.data.AUTOTUNE)

        # `prefetch(tf.data.AUTOTUNE)` è un'ottimizzazione delle performance.
        # Permette alla CPU di preparare il batch successivo di dati mentre la GPU sta elaborando quello corrente.
        # Questo previene i "colli di bottiglia" dovuti al caricamento dei dati.
        return processed_ds.prefetch(tf.data.AUTOTUNE), class_names
    except Exception as e:
        logging.error(f"MAKE_DATASET: Errore durante creazione dataset da {dirpath}: {e}")
        return None, None


# Messaggio di conferma che le definizioni siano state caricate in memoria.
print("Funzioni per Modello e Dataset definite.")

Funzioni per Modello e Dataset definite.


In [ ]:
# Questa funzione incapsula l'intero processo di addestramento del modello.
# Prende un oggetto `args` (che simula gli argomenti da riga di comando)
# contenente tutte le configurazioni necessarie.
def train(args):
    """Esegue il ciclo di training completo."""
    # Inizializza il sistema di logging per questa esecuzione.
    setup_logging(args.output)
    logging.info("Inizio Processo di Training")

    # Preparazione dei Dataset
    # Converte i percorsi da stringa a oggetti Path per una gestione più semplice.
    train_dir_path = Path(args.train_dir)
    # Gestisce il caso in cui un percorso di validazione separato non sia fornito.
    val_dir_path = Path(args.val_dir) if args.val_dir else None

    # Controlla se è stata fornita una directory di validazione separata e se esiste.
    if val_dir_path and val_dir_path.is_dir():
        # Se sì, crea il dataset di training e quello di validazione da due directory distinte.
        logging.info(f"Uso directory di validazione separata: {val_dir_path}")
        # Per il training: mescola i dati (shuffle=True) e applica data augmentation (augment=True).
        train_ds, class_names = make_dataset(train_dir_path, shuffle=True, augment=True)
        # Per la validazione: non mescola (per avere risultati consistenti) e non applicare augmentation.
        val_ds, _ = make_dataset(val_dir_path, shuffle=False) # (** L'underscore `_` ignora i nomi delle classi, che già abbiamo )
    else:
        # Se non c'è una cartella di validazione, crea entrambi i set partendo dalla sola cartella di training,
        # riservando una frazione dei dati (es. 20%) per la validazione.
        logging.info(f"Uso 20% split da {train_dir_path} per validazione.")
        # `subset='training'` e `validation_split=0.2` dicono a `make_dataset` di creare il set di training con l'80% dei dati.
        train_ds, class_names = make_dataset(train_dir_path, shuffle=True, subset='training', validation_split=0.2, augment=True)
        # `subset='validation'` crea il set di validazione con il restante 20%.
        val_ds, _ = make_dataset(train_dir_path, shuffle=False, subset='validation', validation_split=0.2)

    # Controllo di sicurezza: se la creazione di uno dei dataset è fallita, interrompe il training.
    if not all([train_ds, val_ds, class_names]):
        logging.error("Creazione dataset fallita. Training interrotto.")
        return # Esce dalla funzione.

    # Recupera il numero di classi e le stampa per verifica.
    num_classes = len(class_names)
    logging.info(f"Trovate {num_classes} classi: {class_names}")

    # Costruisce l'architettura del modello chiamando la funzione definita in precedenza.
    model = build_model(num_classes)

    # Definizione dei Callbacks
    # I callbacks sono oggetti che eseguono azioni specifiche in vari momenti del training (es. alla fine di ogni epoca).

    # `ModelCheckpoint`: Salva il modello.
    checkpoint_head = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(args.output / "model_head_best.keras"), # Dove salvare il file.
        monitor='val_accuracy',          # Metrica da monitorare per decidere se il modello è "migliore".
        save_best_only=True,             # Salva solo se la metrica monitorata è migliorata.
        mode='max',                      # La 'val_accuracy' deve essere massimizzata.
        verbose=1                        # Stampa un messaggio quando il modello viene salvato.
    )
    # Un secondo checkpoint per la fase di fine-tuning.
    checkpoint_fine = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(args.output / "model_fine_tuned_best.keras"),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    # `EarlyStopping`: Interrompe il training se il modello smette di migliorare.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',              # Monitora la perdita sul set di validazione.
        patience=5,                      # Numero di epoche da attendere senza miglioramenti prima di fermarsi.
        restore_best_weights=True,       # Al termine, ripristina i pesi del modello alla migliore epoca.
        verbose=1                        # Stampa un messaggio quando il training viene interrotto.
    )

    # FASE 1: Training della sola testa del modello
    logging.info(f"FASE 1: Training testa per {args.epochs_head} epoche")
    # Avvia il training vero e proprio.
    model.fit(
        train_ds,                        # Dati di training.
        validation_data=val_ds,          # Dati di validazione per monitorare le performance.
        epochs=args.epochs_head,         # Numero di epoche per questa fase.
        callbacks=[checkpoint_head]      # Applica il callback per salvare il miglior modello di questa fase.
    )

    # Dopo la prima fase, carica esplicitamente il miglior modello salvato dal ModelCheckpoint.
    # Questo assicura che si stia partendo dal miglior stato possibile per il fine-tuning,
    # anche se l'ultima epoca non era la migliore.
    logging.info("Caricamento del miglior modello dal training della testa.")
    model = models.load_model(args.output / "model_head_best.keras")

    # FASE 2: Fine-tuning dell'intero modello
    # `model.layers[0]` si riferisce al primo layer del nostro modello, che è il backbone EfficientNetV2.
    model.layers[0].trainable = True # Scongela i pesi del backbone, rendendoli aggiornabili.

    # Ricompila il modello. Questo è NECESSARIO dopo aver cambiato lo stato `trainable` di un layer.
    # Usiamo un learning rate molto più basso per il fine-tuning.
    model.compile(optimizer=optimizers.Adam(learning_rate=args.learn_full_lr),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    logging.info(f"Modello ricompilato per fine-tuning (LR={args.learn_full_lr})")

    logging.info(f"FASE 2: Fine-tuning per max {args.epochs_fine} epoche")
    # Avvia la seconda fase di training.
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=args.epochs_fine,
        # Usa sia il checkpoint per salvare il miglior modello di questa fase,
        # sia l'early stopping per fermarsi se non ci sono più miglioramenti.
        callbacks=[checkpoint_fine, early_stopping]
    )

    # Salvataggio Finale
    # Al termine di tutto il processo, salva il modello finale (che, grazie a `restore_best_weights`,
    # dovrebbe essere il migliore della fase di fine-tuning) in un percorso standard.
    final_model_path = args.output / "place_model.keras"
    logging.info(f"Salvataggio modello finale in: {final_model_path}")
    model.save(final_model_path)

    # Salva la mappatura tra i nomi delle classi (es. "Cucina") e i loro indici numerici (es. 0).
    # Questo file è FONDAMENTALE per l'inferenza, per poter tradurre l'output del modello (un indice)
    # in un'etichetta leggibile.
    class_indices_path = args.output / "class_indices.csv"
    # Crea un DataFrame pandas e lo salva come CSV.
    pd.DataFrame({"class_name": class_names, "index": list(range(num_classes))}).to_csv(class_indices_path, index=False)
    logging.info(f"Mappatura classi salvata in: {class_indices_path}")
    logging.info("Processo di Training Concluso")

# Messaggio di conferma che la funzione `train` è stata definita.
print("Funzione di Training definita.")

Funzione di Training definita.


In [ ]:
# Blocco di Pulizia Opzionale
# Questo blocco di codice, attualmente non in utilizzo,
# Serve per resettare lo stato del training, eliminando i modelli e i log delle esecuzioni precedenti.
# È utile quando si vuole essere sicuri di iniziare un addestramento completamente da zero.

# print("Pulizia dei vecchi file di modello e log in corso...")
# # Lista dei nomi dei file di modello che vengono creati durante il training.
# old_model_files = ["model_head_best.keras", "model_fine_tuned_best.keras", "place_model.keras"]
# # Itera sulla lista per cancellare ogni file.
# for f_name in old_model_files:
#     # Costruisce il percorso completo del file.
#     # `exists()` controlla se il file esiste prima di provare a cancellarlo, per evitare errori.
#     if (DEFAULT_OUTPUT / f_name).exists():
#         # `.unlink()` è il metodo dell'oggetto Path per cancellare un file.
#         (DEFAULT_OUTPUT / f_name).unlink()
# # Controlla anche l'esistenza del file di log dei file processati.
# if PROCESSED_LOG.exists():
#     # E lo cancella se esiste.
#     PROCESSED_LOG.unlink()
# print("Pulizia completata.")


# Configurazione per il Training
# Qui creiamo un oggetto che conterrà tutti i parametri da passare alla funzione `train`.
# Usiamo `SimpleNamespace` come un modo semplice e veloce per creare un oggetto "contenitore"
# che si comporta in modo simile all'output di `argparse`, permettendoci di accedere ai valori
# con la notazione punto (es. `args.train_dir`).

args_train_config = SimpleNamespace(
    # Passiamo i percorsi delle directory di training e validazione.
    # Vengono convertiti in stringa (`str()`) perché alcune librerie più vecchie
    # potrebbero non accettare direttamente gli oggetti Path. È una buona pratica di compatibilità.
    train_dir=str(DEFAULT_TRAIN_DIR),
    val_dir=str(DEFAULT_VAL_DIR),

    # Il percorso della directory di output.
    output=DEFAULT_OUTPUT,

    # I parametri numerici (iperparametri) definiti nella cella di configurazione globale.
    epochs_head=EPOCHS_HEAD,
    epochs_fine=EPOCHS_FINE,
    learn_head_lr=LEARN_HEAD_LR,
    learn_full_lr=LEARN_FULL_LR
)


# ESECUZIONE DEL TRAINING
try:
    # Chiama la funzione `train`, passandole l'oggetto di configurazione appena creato
    # Da questo momento, il controllo passa alla funzione `train` che eseguirà
    # il caricamento dei dati, la costruzione del modello e le due fasi di addestramento
    train(args_train_config)

# Cattura qualsiasi eccezione (`Exception`) che potrebbe verificarsi durante l'esecuzione
# della funzione `train`. Questo previene un crash improvviso del notebook e fornisce
# informazioni utili per il debug.
except Exception as e:
    # Stampa un messaggio di errore chiaro e conciso.
    print(f"ERRORE INASPETTATO DURANTE IL TRAINING: {e}")
    # Importa il modulo `traceback` per ottenere maggiori dettagli sull'errore.
    import traceback
    # `traceback.print_exc()` stampa la "traccia dello stack", che mostra esattamente
    # in quale punto del codice e attraverso quale catena di chiamate di funzioni
    # si è verificato l'errore. È uno strumento di debug potentissimo.
    traceback.print_exc()

2025-07-21 09:53:23 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/PROGETTO/output/run.log
2025-07-21 09:53:23 INFO: Inizio Processo di Training
2025-07-21 09:53:23 INFO: Uso directory di validazione separata: /content/drive/MyDrive/PROGETTO/dataset/val


Found 6716 files belonging to 2 classes.
Found 321 files belonging to 2 classes.


2025-07-21 09:53:32 INFO: Trovate 2 classi: ['economia_interno', 'stum_interno']
2025-07-21 09:53:34 INFO: Modello EfficientNetV2B0 costruito e compilato per il training della testa.
2025-07-21 09:53:34 INFO: --- FASE 1: Training testa per 10 epoche ---


Epoch 1/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9235 - loss: 0.1719 
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to /content/drive/MyDrive/PROGETTO/output/model_head_best.keras
210/210 ━━━━━━━━━━━━━━━━━━━━ 2426s 11s/step - accuracy: 0.9238 - loss: 0.1714 - val_accuracy: 1.0000 - val_loss: 0.0157
Epoch 2/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0052
Epoch 2: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 1.0000 - loss: 0.0052 - val_accuracy: 1.0000 - val_loss: 0.0064
Epoch 3/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0022
Epoch 3: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 4/10
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0014
Epoch 4: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━

2025-07-21 11:12:51 INFO: Caricamento del miglior modello dal training della testa.
2025-07-21 11:12:52 INFO: Modello ricompilato per fine-tuning (LR=1e-05)
2025-07-21 11:12:52 INFO: FASE 2: Fine-tuning per max 8 epoche


Epoch 1/8
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0061
Epoch 1: val_accuracy improved from -inf to 1.00000, saving model to /content/drive/MyDrive/PROGETTO/output/model_fine_tuned_best.keras
210/210 ━━━━━━━━━━━━━━━━━━━━ 295s 1s/step - accuracy: 1.0000 - loss: 0.0061 - val_accuracy: 1.0000 - val_loss: 0.0135
Epoch 2/8
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0053
Epoch 2: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 1.0000 - val_loss: 0.0121
Epoch 3/8
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0047
Epoch 3: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━━━━━━━━━━ 307s 1s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 0.0107
Epoch 4/8
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0043
Epoch 4: val_accuracy did not improve from 1.00000
210/210 ━━━━━━━━━━━━━

2025-07-21 11:49:10 INFO: Salvataggio modello finale in: /content/drive/MyDrive/PROGETTO/output/place_model.keras
2025-07-21 11:49:11 INFO: Mappatura classi salvata in: /content/drive/MyDrive/PROGETTO/output/class_indices.csv
2025-07-21 11:49:11 INFO: Processo di Training Concluso


In [ ]:
# La funzione `infer` è responsabile di usare il modello addestrato per fare previsioni
# su nuovi dati (immagini e video).
def infer(args):
    """Esegue inferenza su immagini e video."""
    # Imposta il logging per questa esecuzione.
    setup_logging(args.output)
    logging.info("Inizio Processo di Inferenza")

    # Caricamento del Modello e delle Classi
    # Definisce i percorsi del modello salvato e del file CSV con la mappatura delle classi.
    model_path = args.output / "place_model.keras"
    class_indices_path = args.output / "class_indices.csv"

    # Controlla che entrambi i file necessari esistano. Se mancano, l'inferenza non può procedere.
    if not model_path.exists() or not class_indices_path.exists():
        logging.error(f"Modello ({model_path}) o indici classi ({class_indices_path}) non trovati.")
        logging.error("Esegui prima il training per creare il modello.")
        return # Interrompe la funzione.

    try:
        # Carica il modello Keras completo dal file.
        model = models.load_model(model_path)
        # Legge il file CSV delle classi usando pandas.
        class_names_df = pd.read_csv(class_indices_path)
        # Assicura che le classi siano ordinate correttamente secondo il loro indice,
        # poi le estrae in una lista. Questo garantisce che `class_names[0]` corrisponda
        # alla prima classe, `class_names[1]` alla seconda, e così via.
        class_names = class_names_df.sort_values("index")["class_name"].tolist()
        logging.info(f"Modello e {len(class_names)} classi caricate: {class_names}")
    except Exception as e:
        logging.error(f"Errore durante il caricamento del modello o delle classi: {e}")
        return

    # Crea la directory di output dove verranno salvate le copie delle immagini classificate.
    CLASSIFIED_IMG_OUTPUT_DIR = args.output / "classified_images"
    CLASSIFIED_IMG_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    logging.info(f"Le immagini classificate verranno salvate in: {CLASSIFIED_IMG_OUTPUT_DIR}")


    # Preparazione del File CSV di Output
    # Apre il file CSV in modalità 'a' (append), per aggiungere nuove righe senza cancellare le vecchie.
    # `newline=''` è importante per evitare righe vuote extra nel CSV.
    with open(args.csv_path, "a", newline="", encoding='utf-8') as csv_file:
        # Crea un `DictWriter`, che permette di scrivere righe nel CSV usando dizionari,
        # rendendo il codice più leggibile e meno propenso a errori di ordine delle colonne.
        writer = csv.DictWriter(csv_file, fieldnames=["SourceType", "PredictedPlace", "Confidence", "OriginalPath", "OutputPath", "IsVideo", "Sharpness"])
        # Se il file è vuoto (`csv_file.tell() == 0`), scrive l'intestazione con i nomi delle colonne.
        if csv_file.tell() == 0:
            writer.writeheader()

        # Funzione Helper per Preprocessing Immagini
        # Questa funzione interna (nested function) gestisce il caricamento e la preparazione di una singola immagine.
        def load_and_preprocess_img_infer(path_str):
            try:
                # Legge il file immagine dal disco come byte.
                img_bytes = tf.io.read_file(path_str)
                # Decodifica i byte in un tensore di immagine.
                img = tf.image.decode_image(img_bytes, channels=3, expand_animations=False)
                # Ridimensiona l'immagine alla dimensione richiesta dal modello.
                img = tf.image.resize(img, IMG_SIZE, method='bilinear')
                # Applica la stessa funzione di preprocessing usata durante il training.
                img_preprocessed = preprocess_input(img)
                return img_preprocessed
            except Exception as e:
                logging.warning(f"Errore caricamento immagine {path_str}: {e}")
                return None

        # BLOCCO DI INFERENZA SULLE IMMAGINI STATICHE
        logging.info(f"Inizio Inferenza su Immagini Statiche in: {args.img_dir}")
        img_dir = Path(args.img_dir)
        if img_dir.is_dir():
            # Cerca tutti i file con estensioni di immagine comuni nella directory di input.
            image_files = sorted( list(img_dir.glob('*.jpg'))   + list(img_dir.glob('*.jpeg'))  + list(img_dir.glob('*.png'))   + list(img_dir.glob('*.JPG'))   + list(img_dir.glob('*.JPEG'))  + list(img_dir.glob('*.PNG')))
            logging.info(f"Trovate {len(image_files)} immagini da processare.")

            # Itera su ogni immagine trovata, usando `tqdm` per una barra di avanzamento.
            for img_path in tqdm(image_files, desc="Processing Images"):
                # Crea una chiave univoca per l'immagine per il log dei file processati.
                key = f"IMG::{img_path.resolve()}"
                if is_already_processed(key):
                    logging.info(f"Immagine già processata, saltata: {img_path.name}")
                    continue

                # Carica e preprocessa l'immagine.
                img_tensor = load_and_preprocess_img_infer(str(img_path))
                if img_tensor is None:
                    continue # Salta l'immagine se il caricamento fallisce.

                # Il modello si aspetta un batch di immagini, non una singola immagine.
                # `tf.expand_dims` aggiunge una dimensione all'inizio, trasformando la forma
                # da (224, 224, 3) a (1, 224, 224, 3).
                img_batch = tf.expand_dims(img_tensor, axis=0)

                prediction = model.predict(img_batch, verbose=0)[0]
                pred_index = np.argmax(prediction)
                confidence = float(np.max(prediction))

                if confidence >= 0.50:
                    label = class_names[pred_index]
                    logging.info(f"Immagine: {img_path.name} -> Classe: {label} (Conf: {confidence:.3f}) -> ACCETTATA")

                    # Salvataggio e Archiviazione del Risultato
                    dest_dir = CLASSIFIED_IMG_OUTPUT_DIR / label
                    dest_dir.mkdir(parents=True, exist_ok=True)
                    dest_path = dest_dir / img_path.name
                    shutil.copy(str(img_path), str(dest_path))
                    update_image_exif(dest_path, label)

                    # Scrive una riga nel file CSV con tutte le informazioni raccolte.
                    writer.writerow({
                        "SourceType": "Image", "PredictedPlace": label, "Confidence": f"{confidence:.4f}",
                        "OriginalPath": str(img_path.resolve()), "OutputPath": str(dest_path.resolve()),
                        "IsVideo": 0, "Sharpness": "N/A"
                    })
                    # Marca l'immagine come processata per non rielaborarla in futuro.
                    mark_processed(key)
                else:
                    # (Opzionale ma raccomandato) Logga le immagini che vengono scartate
                    label = class_names[pred_index] # Ottieni comunque la label migliore per il log
                    logging.info(f"Immagine: {img_path.name} -> Classe: {label} (Conf: {confidence:.3f}) -> RIFIUTATA (soglia non raggiunta)")
        # FINE BLOCCO IMMAGINI


        # BLOCCO DI INFERENZA SUI VIDEO
        logging.info(f"Inizio Inferenza su Video in: {args.vid_dir} ")
        vid_dir = Path(args.vid_dir)
        if vid_dir.is_dir():
            # Cerca tutti i file video comuni.
            video_files = sorted(list(vid_dir.glob("*.mp4")) + list(vid_dir.glob("*.mov")) + list(vid_dir.glob("*.avi")))
            logging.info(f"Trovati {len(video_files)} video da processare.")

            for vid_path in tqdm(video_files, desc="Processing Videos"):
                key = f"VID::{vid_path.resolve()}"
                if is_already_processed(key):
                    logging.info(f"Video già processato, saltato: {vid_path.name}")
                    continue

                # Estrae i frame dal video.
                temp_video_frame_dir = TEMP_FRAME_EXTRACT_DIR / vid_path.stem
                frame_paths = extract_frames(vid_path, temp_video_frame_dir, args.frame_interval)
                if not frame_paths: continue

                # Preprocessa tutti i frame estratti.
                valid_frames_data = [d for d in [load_and_preprocess_img_infer(str(p)) for p in frame_paths] if d is not None]
                if not valid_frames_data:
                    shutil.rmtree(temp_video_frame_dir, ignore_errors=True) # Pulisce i frame temporanei.
                    continue

                # Esegue la predizione su tutti i frame in un unico batch per efficienza.
                # `tf.stack` converte la lista di tensori in un unico tensore-batch.
                predictions = model.predict(tf.stack(valid_frames_data), batch_size=BATCH_SIZE)
                pred_indices = np.argmax(predictions, axis=1) # Ottiene l'indice predetto per ogni frame.
                pred_confidences = np.max(predictions, axis=1) # Ottiene la confidenza per ogni frame.

                # Aggregazione dei Risultati per il Video
                # `np.bincount` conta le occorrenze di ogni indice. `argmax` trova l'indice più frequente (voto di maggioranza).
                majority_idx = np.bincount(pred_indices).argmax()
                video_label = class_names[majority_idx]
                # Calcola la confidenza media solo per i frame che hanno votato per la classe di maggioranza.
                video_confidence = float(np.mean(pred_confidences[pred_indices == majority_idx]))

                logging.info(f"Video: {vid_path.name} -> Classe: {video_label} (Conf: {video_confidence:.3f})")

                # Selezione e Salvataggio dei Frame Migliori
                candidate_frames_info = []
                # Rivede tutti i frame...
                for i, frame_path in enumerate(frame_paths):
                    # ...e considera solo quelli che appartengono alla classe di maggioranza.
                    if pred_indices[i] == majority_idx:
                        # Calcola la loro nitidezza.
                        sharpness = calculate_sharpness(cv2.imread(str(frame_path)))
                        # Se è sopra la soglia...
                        if sharpness >= args.sharpness_threshold:
                            # ...lo aggiunge alla lista dei candidati.
                            candidate_frames_info.append((sharpness, frame_path, pred_confidences[i]))

                # Ordina i frame candidati in base alla nitidezza, dal più alto al più basso.
                candidate_frames_info.sort(key=lambda x: x[0], reverse=True)
                # Seleziona i migliori N frame.
                best_frames_to_save = candidate_frames_info[:args.frames_per_place_per_video]

                saved_best_frame_paths = []
                final_sharpness = "N/A"
                if best_frames_to_save:
                    # Crea una cartella di output specifica per i frame di questo video.
                    video_best_frames_dir = BEST_FRAMES_OUTPUT_DIR / video_label / vid_path.stem
                    video_best_frames_dir.mkdir(parents=True, exist_ok=True)
                    final_sharpness = f"{best_frames_to_save[0][0]:.1f}"
                    for sharpness, frame_p, conf in best_frames_to_save:
                        # Costruisce un nome di file descrittivo.
                        out_filename = f"{vid_path.stem}__{video_label}__frame{frame_p.stem.split('frame')[-1]}_sharp{sharpness:.0f}.jpg"
                        out_path = video_best_frames_dir / out_filename
                        shutil.copy(str(frame_p), str(out_path))
                        update_image_exif(out_path, video_label)
                        saved_best_frame_paths.append(str(out_path.resolve()))

                # Scrive i risultati aggregati del video nel file CSV.
                writer.writerow({
                    "SourceType": "Video", "PredictedPlace": video_label, "Confidence": f"{video_confidence:.4f}",
                    "OriginalPath": str(vid_path.resolve()), "OutputPath": ";".join(saved_best_frame_paths) if saved_best_frame_paths else "N/A",
                    "IsVideo": 1, "Sharpness": final_sharpness
                })
                mark_processed(key)
                # Pulisce la cartella temporanea dei frame per questo video.
                shutil.rmtree(temp_video_frame_dir, ignore_errors=True)
        else:
            logging.warning(f"La directory dei video {vid_dir} non esiste.")
        # FINE BLOCCO VIDEO

    logging.info("Processo di Inferenza Concluso")

In [ ]:
# Blocco di Pulizia del Log di Processamento
# A differenza del blocco di pulizia nella cella del training, questo è attivo di default.
# Il suo scopo è cancellare il file `processed.log` prima di ogni esecuzione.
# Questo assicura che l'inferenza venga eseguita su TUTTI i file presenti nelle
# cartelle di input, anche se sono già stati analizzati in una sessione precedente.
# Se volessi un comportamento "incrementale" (analizzare solo i file nuovi),
# dovresti commentare o rimuovere questo blocco.

# Definisce il percorso del file di log.
log_file_path = DEFAULT_OUTPUT / "processed.log"
# Controlla se il file esiste.
if log_file_path.exists():
    # Stampa messaggi informativi per l'utente.
    print(f"Cancellazione del vecchio file di log: {log_file_path}")
    # Cancella il file.
    log_file_path.unlink()
    print("Log cancellato. Tutti i file verranno ri-processati.")


# Configurazione per l'Inferenza
# Crea un oggetto `SimpleNamespace` per contenere tutti i parametri da passare
# alla funzione `infer`. Questo raggruppa ordinatamente la configurazione.
args_infer_config = SimpleNamespace(
    # Percorsi delle directory di input per immagini e video.
    img_dir=str(DEFAULT_IMG_DIR),
    vid_dir=str(DEFAULT_VID_DIR),

    # Percorsi di output.
    output=DEFAULT_OUTPUT,
    csv_path=DEFAULT_CSV,

    # Parametri specifici per il processamento dei video.
    frame_interval=FRAME_INTERVAL,
    sharpness_threshold=SHARPNESS_THRESHOLD,
    frames_per_place_per_video=FRAMES_PER_PLACE_PER_VIDEO,
)

# Stampa di Verifica dei Parametri
# Stampa un riepilogo delle configurazioni chiave per l'inferenza.
# È una buona pratica per avere una conferma visiva che lo script
# sta leggendo e scriverà nelle cartelle corrette.
print("\n--- Parametri per l'Inferenza ---")
print(f"Directory Immagini Input: {args_infer_config.img_dir}")
print(f"Directory Video Input: {args_infer_config.vid_dir}")
print(f"Directory Output: {args_infer_config.output}")
print(f"File CSV Risultati: {args_infer_config.csv_path}")

# ESECUZIONE DELL'INFERENZA
try:
    # Chiama la funzione `infer` con la configurazione appena creata.
    # A questo punto, lo script inizierà a caricare il modello e a processare
    # i file uno per uno, come definito nella cella 10.
    infer(args_infer_config)

    # Se la funzione `infer` termina senza errori, stampa un messaggio di successo
    # e un riepilogo di dove trovare i risultati. Questo è molto utile per l'utente.
    print(f"\n--- INFERENZA COMPLETATA ---")
    print(f"Controlla i risultati nel file CSV: {args_infer_config.csv_path}")
    print(f"Le immagini classificate si trovano in: {args_infer_config.output / 'classified_images'}")
    print(f"I frame migliori dei video si trovano in: {BEST_FRAMES_OUTPUT_DIR}")

# Cattura qualsiasi errore imprevisto durante l'inferenza.
except Exception as e:
    # Stampa un messaggio di errore e la traccia completa dello stack per facilitare il debug.
    print(f"ERRORE INASPETTATO DURANTE L'INFERENZA: {e}")
    import traceback
    traceback.print_exc()

2025-07-21 12:40:18 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/PROGETTO/output/run.log
2025-07-21 12:40:18 INFO: Inizio Processo di Inferenza


Cancellazione del vecchio file di log: /content/drive/MyDrive/PROGETTO/output/processed.log
Log cancellato. Tutti i file verranno ri-processati.

--- Parametri per l'Inferenza ---
Directory Immagini Input: /content/drive/MyDrive/PROGETTO/dataset/test
Directory Video Input: /content/drive/MyDrive/PROGETTO/dataset/test
Directory Output: /content/drive/MyDrive/PROGETTO/output
File CSV Risultati: /content/drive/MyDrive/PROGETTO/output/results.csv


2025-07-21 12:40:20 INFO: Modello e 2 classi caricate: ['economia_interno', 'stum_interno']
2025-07-21 12:40:20 INFO: Le immagini classificate verranno salvate in: /content/drive/MyDrive/PROGETTO/output/classified_images
2025-07-21 12:40:20 INFO: Inizio Inferenza su Immagini Statiche in: /content/drive/MyDrive/PROGETTO/dataset/test
2025-07-21 12:40:20 INFO: Trovate 21 immagini da processare.
Processing Images: 100%|██████████| 21/21 [00:26<00:00,  1.28s/it]
2025-07-21 12:40:47 INFO: Inizio Inferenza su Video in: /content/drive/MyDrive/PROGETTO/dataset/test 
2025-07-21 12:40:47 INFO: Trovati 0 video da processare.
Processing Videos: 0it [00:00, ?it/s]
2025-07-21 12:40:47 INFO: Processo di Inferenza Concluso



--- INFERENZA COMPLETATA ---
Controlla i risultati nel file CSV: /content/drive/MyDrive/PROGETTO/output/results.csv
Le immagini classificate si trovano in: /content/drive/MyDrive/PROGETTO/output/classified_images
I frame migliori dei video si trovano in: /content/drive/MyDrive/PROGETTO/output/best_frames


In [ ]:
# NUOVA CELLA - DEFINIZIONE DELLA FUNZIONE DI TEST

def test(args):
    """Esegue la valutazione del modello su un set di dati di test separato."""
    setup_logging(args.output)
    logging.info("--- Inizio Processo di Test ---")

    model_path = args.output / "place_model.keras"
    if not model_path.exists():
        logging.error(f"Modello non trovato in: {model_path}. Esegui prima il training.")
        return

    try:
        model = models.load_model(model_path)
        logging.info(f"Modello caricato da: {model_path}")
    except Exception as e:
        logging.error(f"Errore caricamento modello: {e}")
        return

    test_dir_path = Path(args.test_dir)
    if not test_dir_path.is_dir():
        logging.error(f"Directory di test non trovata: {test_dir_path}")
        return

    test_ds, class_names = make_dataset(test_dir_path, shuffle=False, augment=False)
    if not test_ds:
        logging.error("Creazione del dataset di test fallita.")
        return

    logging.info(f"Dataset di test caricato con classi: {class_names}")
    logging.info("Valutazione del modello sul set di test in corso...")

    results = model.evaluate(test_ds, verbose=1)

    logging.info("--- Risultati del Test ---")
    print("\n--- Risultati del Test ---")
    print(f"Loss sul set di test: {results[0]:.4f}")
    print(f"Accuracy sul set di test: {results[1] * 100:.2f}%")
    logging.info(f"Loss: {results[0]:.4f} - Accuracy: {results[1] * 100:.2f}%")
    logging.info("--- Processo di Test Concluso ---")

print("Funzione di Test definita.")

Funzione di Test definita.


In [ ]:
# NUOVA CELLA - ESECUZIONE DEL TEST - DA CONTROLLARE

args_test_config = SimpleNamespace(
    test_dir=str(DEFAULT_TEST_DIR),
    output=DEFAULT_OUTPUT
)

try:
    print("\nLancio della valutazione sul set di test...")
    test(args_test_config)
except Exception as e:
    print(f"ERRORE INASPETTATO DURANTE IL TEST: {e}")
    import traceback
    traceback.print_exc()

2025-07-21 11:49:26 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/PROGETTO/output/run.log
2025-07-21 11:49:26 INFO: --- Inizio Processo di Test ---



Lancio della valutazione sul set di test...


2025-07-21 11:49:27 INFO: Modello caricato da: /content/drive/MyDrive/PROGETTO/output/place_model.keras


Found 0 files belonging to 0 classes.


2025-07-21 11:49:27 ERROR: MAKE_DATASET: Errore durante creazione dataset da /content/drive/MyDrive/PROGETTO/dataset/test: No images found in directory /content/drive/MyDrive/PROGETTO/dataset/test. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')
2025-07-21 11:49:27 ERROR: Creazione del dataset di test fallita.
